In [1]:
import cv2
import numpy as np

In [2]:
# 图像查找,在train_img(大图)中查找query(某个物体的图片,小图)

In [4]:
'''
关于图像变换矩阵的一些api
estimateRigidTransform()：计算多个二维点对或者图像之间的最优仿射变换矩阵 （2行x3列），H可以是部分自由度，比如各向一致的切变。

getAffineTransform()：计算3个二维点对之间的仿射变换矩阵H（2行x3列），自由度为6.

warpAffine()：对输入图像进行仿射变换

findHomography： 计算多个二维点对之间的最优单映射变换矩阵 H（3行x3列） ，使用最小均方误差或者RANSAC方法 。

getPerspectiveTransform()：计算4个二维点对之间的透射变换矩阵 H（3行x3列）

warpPerspective()： 对输入图像进行透射变换

perspectiveTransform()：对二维或者三维矢量进行透射变换，也就是对输入二维坐标点或者三维坐标点进行投射变换。

estimateAffine3D：计算多个三维点对之间的最优三维仿射变换矩阵H （3行x4列）

transform()：对输入的N维矢量进行变换，可用于进行仿射变换、图像色彩变换.

findFundamentalMat：计算多个点对之间的基矩阵H。
'''
pass

In [5]:
# 计算单应性矩阵的api findHomography 详细说明

# findHomography(srcPoints, dstPoints
#               [, method[, ransacReprojThreshold[, mask[, maxIters[, confidence]]]]]) -> retval, mask

# method 0 -使用所有点的常规方法
#        RANSAC random sample consensus 随机抽样一致性(常用)
#        LMEDS 最小中值鲁棒用法 
#        ...

# ransacReprojThreshold 将点对视为内点的最大允许重投影错误阈值(单位是像素 一般1-10 常用5) 适用于 RANSAC 和 RHO方法

# mask 输出掩码矩阵 通常由 RANSAC 或者 LMEDS 设置, 注意 输入没有设置仅针对输出

# maxIters  RANSAC 迭代次数 默认2000
# confidence 置信度阈值(0-1)

In [6]:
# 读取两张示例用的图片,并对要查找的img做一些变形处理
train_img = cv2.imread('../data/R-C.jpg')
train_img = cv2.resize(train_img,None,fx =0.5,fy=0.5)
train_img_gray = cv2.cvtColor(train_img,cv2.COLOR_BGR2GRAY)

querry_img = cv2.imread('../data/R-C-3.png')
querry_img_gray = cv2.cvtColor(querry_img,cv2.COLOR_BGR2GRAY)


![image.png](004.png)

![image-3.png](005.png)

In [7]:
# 对大图进行透视变换

# 获取变换矩阵 getPerspectiveTransform(src, dst[, solveMethod]) -> retval
src = np.float32([[0,0],[960,0],[0,600],[960,600]])
dst = np.float32([[50,50],[960,0],[0,600],[910,500]])
M = cv2.getPerspectiveTransform(src,dst)
print(M)

# 应用透视变换
train_img_gray = cv2.warpPerspective(train_img_gray,M,(960,600))
train_img =  cv2.warpPerspective(train_img,M,(960,600))

# 现在两张图像如下
outImg =cv2.drawMatches(querry_img,None,train_img,None,None,None)
cv2.imshow('img',outImg)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[ 1.04902778e+00 -8.33333333e-02  5.00000000e+01]
 [-5.20833333e-02  9.22222222e-01  5.00000000e+01]
 [ 1.05324074e-04  9.25925926e-06  1.00000000e+00]]


![image-2.png](006.png)

In [8]:
# 创建检测特征对象
sift = cv2.xfeatures2d.SIFT_create()

# 检测特征
train_kp,train_des = sift.detectAndCompute(train_img_gray,None)
querry_kp,querry_des = sift.detectAndCompute(querry_img_gray,None)

# 进行匹配
bf = cv2.BFMatcher(cv2.NORM_L1,False)
matches = bf.match(querry_des,train_des)
outImg =cv2.drawMatches(querry_img,querry_kp,train_img,train_kp,matches,None)
cv2.imshow('img',outImg)
cv2.waitKey(0)
cv2.destroyAllWindows()

![image.png](007.png)

In [9]:
# 选择匹配距离前50%的
distance_list = [m.distance for m in matches]
idx = np.argsort(np.array(distance_list))
good_idx = idx[0:int(0.5*len(distance_list))]
# print(good_idx)
good_matches = []
for i in good_idx:
    good_matches.append(matches[i])

outImg =cv2.drawMatches(querry_img,querry_kp,train_img,train_kp,good_matches,None)
cv2.imshow('img',outImg)
cv2.waitKey(0)
cv2.destroyAllWindows()
# 可以看到,有些错误,但主体是正确的

![image.png](008.png)

In [10]:
# 计算单应性矩阵最少要求4个点(透射变换矩阵相当于是特例正好4个点)
if len(good_matches)>=4:
    train_point = np.float32([train_kp[m.trainIdx].pt for m in good_matches]).reshape(-1,1,2)
    querry_point = np.float32([querry_kp[m.queryIdx].pt for m in good_matches]).reshape(-1,1,2)
    
    # 根据匹配的点计算单应性矩阵
    H,_= cv2.findHomography(querry_point,train_point,cv2.RANSAC,5)
    '''
    findHomography(srcPoints, dstPoints
    [, method[, ransacReprojThreshold[, mask[, maxIters[, confidence]]]]]) -> retval, mask
    '''
    print(H)
    # 通过单应性矩阵计算小图上的点在大图上的位置,这里用的就是小图的4个边界点
    h,w,_ = querry_img.shape
    p = np.float32([[[0,0],[0,h-1],[w-1,h-1],[w-1,0]]]) # 小图的四个边界点(左上,左下,右下,右上) 1*n*2 格式
    print(p.shape) # (1, 4, 2)
    #cv2.warpPerspective 注意不是使用这个接口,这个接口是用来对图像进行变换,需要的是变换后的坐标
    new_p = cv2.perspectiveTransform(p,H)
    print(new_p,'\n',new_p.shape) #  (1, 4, 2)
    
    # 在大图中train_img 把这四个点的多边形绘制出来
    cv2.polylines(train_img,[np.int64(new_p)],isClosed=True,color = (0,0,255),thickness=2)
#     cv2.imshow('ex',train_img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    outImg =cv2.drawMatches(querry_img,querry_kp,train_img,train_kp,good_matches,None)
    cv2.imshow('img',outImg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
else:
    print('没有足够的特征点匹配关系')

[[ 9.60616454e-01 -6.16994015e-02  2.37441826e+02]
 [-1.33774447e-01  9.20558700e-01  4.37164258e+02]
 [-8.65806530e-05  7.47751014e-05  1.00000000e+00]]
(1, 4, 2)
[[[237.44183 437.16425]
  [225.5782  569.7357 ]
  [337.15643 560.1225 ]
  [350.41074 426.11984]]] 
 (1, 4, 2)


![image.png](009.png)